In [1]:
import os
import faiss
import numpy as np
from tqdm import tqdm
from sentence_transformers import SentenceTransformer

## Этап 1. Загрузка текстов из файлов

In [2]:
TEXT_FILES_DIR = "garant"

In [3]:
def load_texts_from_folder(folder_path):
    texts = []
    filenames = []
    for file in tqdm(sorted(os.listdir(folder_path)), desc="Загрузка текстов"):
        if file.endswith(".txt"):
            file_path = os.path.join(folder_path, file)
            with open(file_path, "r", encoding="utf-8") as f:
                text = f.read().strip()
                texts.append(text)
                filenames.append(file)
    return texts, filenames

In [4]:
texts, filenames = load_texts_from_folder(TEXT_FILES_DIR)

Загрузка текстов: 100%|█████████████████████████████████████████████████████████████| 1628/1628 [00:16<00:00, 96.10it/s]


## Этап 2. Определение функций для разбиения и агрегации эмбеддингов

In [5]:
# Функция для разбиения текста на чанки по количеству символов, не разрывая слова
def split_text_by_length(text, max_length=1000):
    words = text.split()
    chunks = []
    current_chunk = ""
    for word in words:
        # Если добавление слова не превышает лимит, добавляем его
        if len(current_chunk) + len(word) + 1 <= max_length:
            current_chunk = current_chunk + " " + word if current_chunk else word
        else:
            chunks.append(current_chunk)
            current_chunk = word
    if current_chunk:
        chunks.append(current_chunk)
    return chunks

Если документ длинный, он разбивается на чанки, для каждого чанка вычисляется эмбеддинг, затем эмбеддинги усредняются

In [6]:
# Функция для вычисления агрегированного эмбеддинга документа
def get_document_embedding(text, model, chunk_size=1000):
    if len(text) <= chunk_size:
        return model.encode(text)
    else:
        chunks = split_text_by_length(text, max_length=chunk_size)
        chunk_embeddings = model.encode(chunks, convert_to_numpy=True)
        return np.mean(chunk_embeddings, axis=0)

## Этап 3. Вычисление эмбеддингов документов с учетом разбиения

In [7]:
MODEL_NAME = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
model = SentenceTransformer(MODEL_NAME)

In [8]:
document_embeddings = []
for text in tqdm(texts, desc="Обработка документов"):
    emb = get_document_embedding(text, model, chunk_size=1000)
    document_embeddings.append(emb)
document_embeddings = np.array(document_embeddings)

print(f"\nФорма эмбеддингов: {document_embeddings.shape}")

Обработка документов: 100%|█████████████████████████████████████████████████████████| 1628/1628 [00:52<00:00, 31.23it/s]


Форма эмбеддингов: (1628, 384)


## Этап 4: Создание FAISS-индекса

FAISS используется для быстрого поиска среди эмбеддингов

In [9]:
dimension = document_embeddings.shape[1]

Создаём FAISS-индекс

In [10]:
index = faiss.IndexFlatL2(dimension)
index.add(document_embeddings.astype('float32'))

Сохраняем индекс для последующего использования

In [11]:
faiss.write_index(index, "results_BERT_UpgradeChunks/text_index.faiss")
np.save("results_BERT_UpgradeChunks/filenames.npy", np.array(filenames))  # сохраняем список имён файлов

## Этап 5: Функция поиска похожих текстов

In [12]:
def find_similar_texts(query, top_k=5):
    query_embedding = model.encode([query], convert_to_numpy=True)
    distances, indices = index.search(query_embedding.astype('float32'), top_k)
    
    print("Наиболее похожие тексты:")
    for i, idx in enumerate(indices[0]):
        print(f"{i+1}. {filenames[idx]} (дистанция: {distances[0][i]:.4f})")
        print(texts[idx][:300] + "...") # выводим первые 300 символов для примера
        print("-" * 80)

In [13]:
query_text = "Международный день инвалидов в Хабаровске"
find_similar_texts(query_text, top_k=20)

Наиболее похожие тексты:
1. garant_1261.txt (дистанция: 14.0135)
﻿
Постановление Мэра г. Хабаровска
от 11 ноября 2004 г. N 1650
"О проведении Международного дня инвалидов в городе Хабаровске в 2004 году"

В связи с проведением 3 декабря 2004 года Международного дня инвалидов и в целях привлечения внимания широких слоев общественности города, предприятий, учрежден...
--------------------------------------------------------------------------------
2. garant_0055.txt (дистанция: 14.6030)
﻿
Постановление Губернатора Ярославской области
от 21 декабря 2004 г. N 859
"О квотировании рабочих мест для трудоустройства инвалидов"

В соответствии с Федеральным законом от 24.11.95 N 181-ФЗ "О социальной защите инвалидов в Российской Федерации" и в целях установления гарантии трудовой занятости...
--------------------------------------------------------------------------------
3. garant_1851.txt (дистанция: 14.8375)
﻿
Решение Казанского Совета народных депутатов
от 1 октября 2004 г. N 22-20
"О мерах 